In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

# For model selection
from sklearn.model_selection import train_test_split

# Other
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
import re



In [2]:
# Loading the datasets
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
df = pd.concat([df_train,df_test],axis=0)

In [4]:
df.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,NaN,3,1Fam,TA,No,706.0,0.0,...,WD,0,Pave,8,856.0,AllPub,0,2003,2003,2008
1,1262,0,0,NaN,3,1Fam,TA,Gd,978.0,0.0,...,WD,0,Pave,6,1262.0,AllPub,298,1976,1976,2007
2,920,866,0,NaN,3,1Fam,TA,Mn,486.0,0.0,...,WD,0,Pave,6,920.0,AllPub,0,2001,2002,2008
3,961,756,0,NaN,3,1Fam,Gd,No,216.0,0.0,...,WD,0,Pave,7,756.0,AllPub,0,1915,1970,2006
4,1145,1053,0,NaN,4,1Fam,TA,Av,655.0,0.0,...,WD,0,Pave,9,1145.0,AllPub,192,2000,2000,2008


In [5]:
df.shape

(2919, 81)

In [6]:
df.tail()

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
1454,546,546,0,NaN,3,Twnhs,TA,No,0.0,0.0,...,WD,0,Pave,5,546.0,AllPub,0,1970,1970,2006
1455,546,546,0,NaN,3,TwnhsE,TA,No,252.0,0.0,...,WD,0,Pave,6,546.0,AllPub,0,1970,1970,2006
1456,1224,0,0,NaN,4,1Fam,TA,No,1224.0,0.0,...,WD,0,Pave,7,1224.0,AllPub,474,1960,1996,2006
1457,970,0,0,NaN,3,1Fam,TA,Av,337.0,0.0,...,WD,0,Pave,6,912.0,AllPub,80,1992,1992,2006
1458,996,1004,0,NaN,3,1Fam,TA,Av,758.0,0.0,...,WD,0,Pave,9,996.0,AllPub,190,1993,1994,2006


In [7]:
dfd = df

In [8]:
dfd.columns = dfd.columns.map(str.lower)

In [9]:
# Filled alley null values with string 'No'
dfd.alley.fillna('No',inplace=True)

In [10]:
dfd.bsmtcond.fillna('No',inplace=True)

In [11]:
dfd.bsmtexposure.fillna('No Basement',inplace=True)

In [12]:
dfd.bsmtfinsf1.fillna(0.0,inplace=True)

In [13]:
dfd.bsmtfinsf2.fillna(0.0,inplace=True)

In [14]:
dfd.bsmtfintype1.fillna('No Basement',inplace=True)
dfd.bsmtfintype2.fillna('No Basement',inplace=True)

In [17]:
dfd.bsmtfullbath.fillna(0.0,inplace=True)

In [19]:
dfd.bsmthalfbath.fillna(0.0,inplace=True)

In [22]:
dfd.bsmtqual

0       Gd
1       Gd
2       Gd
3       TA
4       Gd
        ..
1454    TA
1455    TA
1456    TA
1457    Gd
1458    Gd
Name: bsmtqual, Length: 2919, dtype: object

In [49]:
dfd.loc[dfd.bsmtfinsf2.isna()]

,1stflrsf,2ndflrsf,3ssnporch,alley,bedroomabvgr,bldgtype,bsmtcond,bsmtexposure,bsmtfinsf1,bsmtfinsf2,...,saletype,screenporch,street,totrmsabvgrd,totalbsmtsf,utilities,wooddecksf,yearbuilt,yearremodadd,yrsold
660,896,0,0,No,2,1Fam,No,No Basement,0.0,NaN,...,ConLD,0,Pave,4,NaN,AllPub,0,1946,1950,2008


In [23]:
obj = dfd.isnull().sum()
for key,value in obj.iteritems():
    print(key,",",value)

1stflrsf , 0
2ndflrsf , 0
3ssnporch , 0
alley , 0
bedroomabvgr , 0
bldgtype , 0
bsmtcond , 0
bsmtexposure , 0
bsmtfinsf1 , 0
bsmtfinsf2 , 0
bsmtfintype1 , 0
bsmtfintype2 , 0
bsmtfullbath , 0
bsmthalfbath , 0
bsmtqual , 81
bsmtunfsf , 1
centralair , 0
condition1 , 0
condition2 , 0
electrical , 1
enclosedporch , 0
extercond , 0
exterqual , 0
exterior1st , 1
exterior2nd , 1
fence , 2348
fireplacequ , 1420
fireplaces , 0
foundation , 0
fullbath , 0
functional , 2
garagearea , 1
garagecars , 1
garagecond , 159
garagefinish , 159
garagequal , 159
garagetype , 157
garageyrblt , 159
grlivarea , 0
halfbath , 0
heating , 0
heatingqc , 0
housestyle , 0
id , 0
kitchenabvgr , 0
kitchenqual , 1
landcontour , 0
landslope , 0
lotarea , 0
lotconfig , 0
lotfrontage , 486
lotshape , 0
lowqualfinsf , 0
mssubclass , 0
mszoning , 4
masvnrarea , 23
masvnrtype , 24
miscfeature , 2814
miscval , 0
mosold , 0
neighborhood , 0
openporchsf , 0
overallcond , 0
overallqual , 0
paveddrive , 0
poolarea , 0
poolqc , 29